# Create Tiff file from Petrel data using Python Tool Pro

In [1]:
from cegalprizm.scripting_server import WorkflowDescription

In [2]:
# Start: PWR Description



pwr_description = WorkflowDescription(name="Export Geotiff files",
                                      category="GIS",
                                      description="Use this workflow to create a Geotiff file of a seismic horizon attribute",
                                      authors="sebastian.grebe@cegal.com",
                                      version="0.1")


pwr_description.add_string_parameter("string_value", "String Value2", "A string input", "hello")
pwr_description.add_object_ref_parameter(name='horizon_interpretation',label='Seismic horizon',description='Select a seismic horizon',object_name='horizon_interpretation')



# End: PWR Description


In [ ]:
if 'parameters' not in locals() and 'parameters' not in globals():
    parameters = pwr_description.get_default_parameters()

In [ ]:
import geopandas as gpd
from geocube.api.core import make_geocube

In [ ]:
from cegalprizm.pythontool import PetrelConnection

ptp = PetrelConnection()

petrel_objects = ptp.get_petrelobjects_by_guids([parameters['horizon_interpretation']])

In [ ]:
var=petrel_objects[0]
myvar2=[i for i in var.horizon_interpretation_3ds][0]
print(myvar2.petrel_name)
myvar3=[i for i in myvar2.horizon_property_3ds][0]
print(myvar3.petrel_name)

In [ ]:
df=myvar3.all().as_dataframe()
df=df.dropna(axis=0,subset='Value')

In [ ]:
xyzs=myvar3.ijks_to_positions([df.I,df.J])
df['X']=xyzs[0]
df['Y']=xyzs[1]
df['Z']=xyzs[2]

In [ ]:
i_step=myvar3.retrieve_stats().get('Inline interval')
print('Inline interval is '+i_step)
x_step=myvar3.retrieve_stats().get('Crossline interval')
print('Crossline interval is '+x_step)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df['X'], y=df['Y'],crs="EPSG:23031"))

In [ ]:
horizon=make_geocube(gdf,measurements=["Value"],resolution=(i_step,x_step))

In [ ]:
filename=var.petrel_name+'_'+myvar2.petrel_name+'_'+myvar3.petrel_name

In [ ]:
horizon.Value.rio.to_raster('D:\Data\PTP_CRS\\'+filename+'.tif')